In [21]:
# Método para resolver las energías de un sistema cuántico
import numpy as np
from sympy import *
from sympy import init_printing; init_printing(use_latex = 'mathjax')
import matplotlib.pyplot as plt

In [22]:
# Variables utilizadas
var('x l m hbar w')

(x, l, m, h̅, w)

In [23]:
n = int(input('Introduce_el_número_de_funciones_que_vas_a_utilizar_para_resolver_el_problema:'))
# Lista para ingresar las funciones
f=[]
# Matriz con integral de solapamiento
Sm=[]
# Matriz con integral de intercambio
Hm=[]
# Aproximación de las energías
for i in range(n):
    f.append((input('Introduce la función %d:' %(i+1))))
f=sympify(f)

# Para partícula en un pozo de potencial de 0 a l
for i in range(n):
    for j in range(n):
        integrando1=(-hbar**2)/(2*m)*(f[i])*diff(f[j],x,2)
        integrando2=(f[i])*f[j]
        c=Integral(integrando1,(x,0,l))
        c=c.doit()
        Hm.append(c)
        d=Integral(integrando2,(x,0,l))
        d=d.doit()
        Sm.append(d)
Sm=np.reshape(Sm,(n,n))
Hm=np.reshape(Hm,(n,n))

# Matriz M: (Hij-Sij)*w
M=(Hm-Sm*w)
H=sympify(Matrix(M))
Hdet=H.det()
# Resolver el determinante para encontrar las energías
E=solve(Hdet,w)
Eev=np.zeros(n)
a=0
# Se evaluan las energías divididas entre 4*pi con los valores de hbar, m=1 y l=10A
for i in range(n):
    a=E[i]/(4*pi)
    Eev[i]=(a.subs({hbar:1.0545718e-34,m:1,l:10e-10})).evalf()


Introduce_el_número_de_funciones_que_vas_a_utilizar_para_resolver_el_problema:4
Introduce la función 1:x*(l-x)
Introduce la función 2:x**2*(l-x)**2
Introduce la función 3:x*(l-x)*(1/2*l-x)
Introduce la función 4:x**2*(l-x)**2*(1/2*l-x)


In [37]:
sympify(Matrix(Sm))

⎡ 5     7             ⎤
⎢l     l              ⎥
⎢──   ───   0      0  ⎥
⎢30   140             ⎥
⎢                     ⎥
⎢  7    9             ⎥
⎢ l    l              ⎥
⎢───  ───   0      0  ⎥
⎢140  630             ⎥
⎢                     ⎥
⎢            7     9  ⎥
⎢           l     l   ⎥
⎢ 0    0   ───   ──── ⎥
⎢          840   5040 ⎥
⎢                     ⎥
⎢            9     11 ⎥
⎢           l     l   ⎥
⎢ 0    0   ────  ─────⎥
⎣          5040  27720⎦

In [38]:
sympify(Matrix(Hm))

⎡  2  3    2  5                ⎤
⎢h̅ ⋅l   h̅ ⋅l                 ⎥
⎢──────  ──────    0       0   ⎥
⎢ 6⋅m     30⋅m                 ⎥
⎢                              ⎥
⎢  2  5    2  7                ⎥
⎢h̅ ⋅l   h̅ ⋅l                 ⎥
⎢──────  ──────    0       0   ⎥
⎢ 30⋅m   105⋅m                 ⎥
⎢                              ⎥
⎢                  2  5    2  7⎥
⎢                h̅ ⋅l   h̅ ⋅l ⎥
⎢  0       0     ──────  ──────⎥
⎢                 40⋅m   280⋅m ⎥
⎢                              ⎥
⎢                  2  7    2  9⎥
⎢                h̅ ⋅l   h̅ ⋅l ⎥
⎢  0       0     ──────  ──────⎥
⎣                280⋅m   1260⋅m⎦

In [31]:
H

⎡  2  3    5      2  5    7                                 ⎤
⎢h̅ ⋅l    l ⋅w  h̅ ⋅l    l ⋅w                               ⎥
⎢────── - ────  ────── - ────        0              0       ⎥
⎢ 6⋅m      30    30⋅m    140                                ⎥
⎢                                                           ⎥
⎢  2  5    7      2  7    9                                 ⎥
⎢h̅ ⋅l    l ⋅w  h̅ ⋅l    l ⋅w                               ⎥
⎢────── - ────  ────── - ────        0              0       ⎥
⎢ 30⋅m    140   105⋅m    630                                ⎥
⎢                                                           ⎥
⎢                                2  5    7      2  7    9   ⎥
⎢                              h̅ ⋅l    l ⋅w  h̅ ⋅l    l ⋅w ⎥
⎢      0              0        ────── - ────  ────── - ──── ⎥
⎢                               40⋅m    840   280⋅m    5040 ⎥
⎢                                                           ⎥
⎢                                2  7    9      2  9    11  ⎥
⎢       

In [39]:
E

⎡    2                   2                  2                   2            ⎤
⎢2⋅h̅ ⋅(-√133 + 14)  2⋅h̅ ⋅(√133 + 14)  2⋅h̅ ⋅(-9⋅√5 + 30)  2⋅h̅ ⋅(9⋅√5 + 30)⎥
⎢──────────────────, ─────────────────, ──────────────────, ─────────────────⎥
⎢        2                   2                  2                   2        ⎥
⎣       l ⋅m                l ⋅m               l ⋅m                l ⋅m      ⎦

In [40]:
for i in range (n):
    print(E[i].evalf())

4.93487481065841*hbar**2/(l**2*m)
51.0651251893416*hbar**2/(l**2*m)
19.7507764050038*hbar**2/(l**2*m)
100.249223594996*hbar**2/(l**2*m)
